In [ ]:
import numpy as np
import torch
import pickle
from Dataset import OntologyDataset
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm
import re
from torch.utils.data.sampler import RandomSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import os

info_path = "dfalc_data"
file_name = "toyOntology.owl"
params = {
        "conceptPath": os.path.join(info_path,file_name+"_concepts.txt"),
        "rolePath": os.path.join(info_path,file_name+"_roles.txt"),
        "individualPath": os.path.join(info_path,file_name+"_individuals.txt"),
        "normalizationPath": os.path.join(info_path,file_name+"_normalization.txt"),
        "batchSize": 3,
        "epochSize":10,
        "earlystopping":10,
        "dist": "minkowski",
        "norm":1,
        "norm_rate":0.5,
        "norm_rate2":0
    }
to_train = False

save_path = "dfalc_data"
if to_train: save_path = os.path.join(save_path,"training")
else: save_path = os.path.join(save_path,"testing")
save_path += "/toyOntology_"
dataset = OntologyDataset(params,save_path)

individualSize = 8


print(dataset.concept2id)
print(dataset.role2id)

In [ ]:
#exists NF3
cEmb_init = torch.zeros(dataset.conceptSize, individualSize)
rEmb_init = torch.zeros(4, individualSize, individualSize)
cEmb_init[2] = torch.FloatTensor([0,0,0,0.9,0.9,0,0,0.9])
cEmb_init[5] = torch.FloatTensor([0.9,0,0,0,0,0,0.9,0])
rEmb_init[3,0,1], rEmb_init[3,2,3], rEmb_init[3,4,5], rEmb_init[3,6,7] = 0.9,0.9,0.9,0

In [ ]:
#forall NF4
cEmb_init[3] = torch.FloatTensor([0,0,0,0.9,0.9,0,0,0.9])
cEmb_init[7] = torch.FloatTensor([0.9,0,0,0,0,0,0.9,0])
rEmb_init[2,0,1], rEmb_init[2,2,3], rEmb_init[2,4,5], rEmb_init[2,6,7] = 0.9,0.9,0.9,0

In [ ]:
#exists NF5
cEmb_init[0] = torch.FloatTensor([0,0,0,0.9,0.9,0,0,0.9])
cEmb_init[4] = torch.FloatTensor([0.9,0,0,0,0,0,0.9,0])
rEmb_init[0,0,1], rEmb_init[0,2,3], rEmb_init[0,4,5], rEmb_init[0,6,7] = 0.9,0.9,0.9,0

In [ ]:
#forall NF6
cEmb_init[1] = torch.FloatTensor([0,0,0,0.9,0.9,0,0,0.9])
cEmb_init[6] = torch.FloatTensor([0.9,0,0,0,0,0,0.9,0])
rEmb_init[1,0,1], rEmb_init[1,2,3], rEmb_init[1,4,5], rEmb_init[1,6,7] = 0.9,0.9,0.9,0

In [25]:

import numpy as np
import torch
import torch.nn as nn

    
class DFALC(nn.Module):
    def __init__(self, params, conceptSize, roleSize, cEmb_init, rEmb_init,  device, name="Godel"):
        super().__init__()
        self.params = params
        self.conceptSize, self.roleSize = conceptSize, roleSize
        self.device = device
        self.cEmb = nn.Parameter(torch.tensor(cEmb_init))
        self.rEmb = nn.Parameter(torch.tensor(rEmb_init))
        self.relu = torch.nn.ReLU()
        # self.c_mask, self.r_mask = self.get_mask()
        self.logic_name = name
        self.epsilon = 1e-2
        self.p=2
        self.alpha=0.8

    def to_sparse(self, A):
        return torch.sparse_coo_tensor(np.where(A!=0),A[np.where(A!=0)],A.shape)
    
    def index_sparse(self, A, idx):
        return torch.where(A.indices[0] in idx)
    
    def pi_0(self, x):
        return (1-self.epsilon)*x+self.epsilon
    
    def pi_1(self, x):
        return (1-self.epsilon)*x
    
    
    def neg(self, x, negf):
        negf = negf.unsqueeze(1)
        # print("negf: ",negf.shape)
        # print("x: ",x.shape)
        negf2 = negf*(-2) + 1
        # print("negf2: ",negf2)
        # print("negf2: ",negf2.shape)
        
        return negf2*x
        
    def t_norm(self, x, y):
        if self.logic_name == "Godel":
            return torch.minimum(x,y)
        elif self.logic_name == "LTN":
            return self.pi_0(x)*self.pi_0(y)
        # elif self.logic_name == "Product":
        #     return x*y
        
    def t_cnorm(self, x, y):
        if self.logic_name == "Godel":
            return torch.maximum(x,y)
        elif self.logic_name == "LTN":
            a = self.pi_1(x)
            b = self.pi_1(y)
            return a+b-a*b
        # elif self.logic_name == "Product":
        #     return x+y-x*y

    def forall(self, r, x):
        if self.logic_name == "Godel":
            return torch.min(self.t_cnorm(1-r,x.unsqueeze(1).expand(r.shape)),2).values
        elif self.logic_name == "LTN":
            return 1-torch.pow(torch.mean(torch.pow(1-self.pi_1(self.t_cnorm(r,x.unsqueeze(1).expand(r.shape))),self.p),2),1/self.p)
        # elif self.logic_name == "Product":
        #     return torch.prod(torch.max(-b,0),2)
    
    def exist(self, r, x):
        if self.logic_name == "Godel":
            return torch.max(self.t_norm(r,x.unsqueeze(1).expand(r.shape)),2).values
        elif self.logic_name == "LTN":
            return torch.pow(torch.mean(torch.pow(self.pi_0(self.t_norm(r,x.unsqueeze(1).expand(r.shape))),self.p),2),1/self.p)
    
    def L2(self, x, dim=1):
        return torch.sqrt(torch.sum((x)**2, dim))
    
    def L2_dist(self, x, y, dim=1):
        return torch.sqrt(torch.sum((x-y)**2, dim))
    
    def L1(self,x,dim=1):
        return torch.sum(torch.abs(x),dim)
    
    def L1_dist(self,x,y,dim=1):
        return torch.sum(torch.abs(x-y),dim)
    
    def HierarchyLoss(self, lefte, righte):
        return torch.mean(self.L1(self.relu(lefte-righte)))


        
    def forward(self, batch, atype, device):
        left, right, negf = batch
        # print("here negf: ", negf.shape)
        # print('here left: ',left.shape)
        # print("here right: ", right.shape)
        
        loss, lefte, righte, loss1 = None, None, None, None
        
        self.cEmb[-1,:].detach().masked_fill_(self.cEmb[-1,:].gt(0.0),1.0)
        self.cEmb[-2,:].detach().masked_fill_(self.cEmb[-2,:].lt(1),0.0)
        
        
        if atype == 0:
            lefte = self.neg(self.cEmb[left],-negf[:,0])
            righte = self.neg(self.cEmb[right],negf[:,1])
            shape = lefte.shape
            # b_c_mask = self.c_mask[left] 
            
        elif atype == 1:
            righte = self.neg(self.cEmb[right], negf[:,2])
            shape = righte.shape
            lefte = self.t_norm(self.neg(self.cEmb[left[:,0]],negf[:,0]), self.neg(self.cEmb[left[:,1]],negf[:,1]))
            loss1 = -righte*(self.relu(lefte-righte).detach())
            
        elif atype == 2:
            lefte = self.neg(self.cEmb[left], negf[:,0])
            shape = lefte.shape
            righte = self.t_cnorm(self.neg(self.cEmb[right[:,0]],negf[:,1]), self.neg(self.cEmb[right[:,1]],negf[:,2]))
            loss1 = -lefte*(self.relu(lefte-righte).detach())

        elif atype == 3:
            lefte = self.neg(self.cEmb[left], negf[:,0])
            shape = lefte.shape
            righte = self.exist(self.rEmb[right[:,0]], self.neg(self.cEmb[right[:,1]],negf[:,1]))

        elif atype == 4:
            lefte = self.neg(self.cEmb[left], negf[:,0])
            shape = lefte.shape
            righte = self.forall(self.rEmb[right[:,0]],self.neg(self.cEmb[right[:,1]], negf[:,1]))
            
            
        elif atype == 5:
            righte = self.neg(self.cEmb[right], negf[:,1])
            shape = righte.shape
            lefte = self.exist(self.rEmb[left[:,0]],self.neg(self.cEmb[left[:,1]], negf[:,0]))
            lefte2 = self.neg(self.cEmb[left[:,1]], negf[:,0])
            righte2 = torch.matmul(righte, self.rEmb[left[:,0]]).squeeze(2)
            # righte1 = torch.matmul(self.rEmb[left[:,0]],lefte2.T).squeeze(2)
            righte1 = torch.bmm(self.rEmb[left[:,0]],lefte2.unsqueeze(2)).squeeze()
            # print("hehre: ",((1-self.relu(righte-self.alpha))*self.relu(righte1-righte)).detach())
            loss1 =  (1-righte)*(((1-self.relu(righte-self.alpha))*self.relu(righte1-self.alpha)*self.relu(self.alpha-righte)).detach()) #self.relu(torch.max(self.rEmb[left[:,0]],1).values-self.alpha)*

        elif atype == 6:
            righte = self.neg(self.cEmb[right], negf[:,1])
            shape = righte.shape
            lefte = self.forall(self.rEmb[left[:,0]],self.neg(self.cEmb[left[:,1]], negf[:,0]))
            lefte2 = self.neg(self.cEmb[left[:,1]], negf[:,0])
            # righte2 = torch.matmul(righte, self.rEmb[left[:,0]]).squeeze(2)
            righte2 = torch.bmm(righte.unsqueeze(1), self.rEmb[left[:,0]]).squeeze()
            loss1 = (1-lefte2)*(((1-self.relu(lefte2-self.alpha))*self.relu(righte2-self.alpha)*self.relu(self.alpha-lefte2)).detach())
        # print("lefte: ", lefte)
        # print("righte: ", righte)
        # print("r: ", torch.max(self.rEmb[left[:,0]],1).values)
        # print("lefte2: ", lefte2)
        # print("righte2: ", righte2)
        # print("righte1: ", righte1)
        # print("loss1: ", loss1)
        # print("atype: ",atype)
        loss = self.HierarchyLoss(lefte, righte)
        return loss#torch.mean(torch.sum(loss1,1))
        # print("loss: ", self.relu(lefte-righte)+loss1)
            
        # return torch.mean(torch.sum(self.relu(lefte-righte),1))

In [26]:
import torch.optim as optim
device = torch.device("cpu")
model = DFALC(params,dataset.conceptSize, dataset.roleSize, cEmb_init=cEmb_init, rEmb_init=rEmb_init, device=device)
dataset.mode = 3
optimizer = optim.Adam(model.parameters(),lr=0.05)
print('human', model.cEmb[5])
print('grass',model.cEmb[2])
print(model.rEmb[3])
for e in range(30):
    for l,r,n in dataset:
        print(l,r,n)
        loss = model([l.unsqueeze(0),r.unsqueeze(0),n.unsqueeze(0)],dataset.mode,device)
        print("loss: ", loss)
        if loss.item() == 0: break
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if loss.item() == 0: break
print('human', model.cEmb[5])
print('grass',model.cEmb[2])
print(model.rEmb[3])
    
    

human tensor([0.9000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9000, 0.0000],
       grad_fn=<SelectBackward>)
grass tensor([0.0000, 0.0000, 0.0000, 0.9000, 0.9000, 0.0000, 0.0000, 0.9000],
       grad_fn=<SelectBackward>)
tensor([[0.0000, 0.9000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.9000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<SelectBackward>)
tensor(5) tensor([3, 2]) tensor([0, 0])
loss:  tensor(1.8000, grad_fn=<MeanBackward0>)
tensor(5) tensor([3, 2]) tensor([0, 0])
loss:  tensor(1.7000, g

c:\Users\92803\.conda\envs\rl_hw\lib\site-packages\ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':
c:\Users\92803\.conda\envs\rl_hw\lib\site-packages\ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


In [27]:
import torch.optim as optim
device = torch.device("cpu")
model = DFALC(params,dataset.conceptSize, dataset.roleSize, cEmb_init=cEmb_init, rEmb_init=rEmb_init, device=device)
dataset.mode = 4
optimizer = optim.Adam(model.parameters(),lr=0.05)
print('teacher', model.cEmb[7])
print('class',model.cEmb[3])
print(model.rEmb[2])
for e in range(30):
    for l,r,n in dataset:
        loss = model([l.unsqueeze(0),r.unsqueeze(0),n.unsqueeze(0)],dataset.mode,device)
        print("loss: ", loss)
        # if loss.item() == 0: break
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # if loss.item() == 0: break
print('teacher', model.cEmb[7])
print('class',model.cEmb[3])
print(model.rEmb[2])
    
    

teacher tensor([0.9000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9000, 0.0000],
       grad_fn=<SelectBackward>)
class tensor([0.0000, 0.0000, 0.0000, 0.9000, 0.9000, 0.0000, 0.0000, 0.9000],
       grad_fn=<SelectBackward>)
tensor([[0.0000, 0.9000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.9000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<SelectBackward>)
loss:  tensor(0.8000, grad_fn=<MeanBackward0>)
loss:  tensor(0.7000, grad_fn=<MeanBackward0>)
loss:  tensor(0.6000, grad_fn=<MeanBackward0>)
loss:  

c:\Users\92803\.conda\envs\rl_hw\lib\site-packages\ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':
c:\Users\92803\.conda\envs\rl_hw\lib\site-packages\ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


In [28]:
model.cEmb

Parameter containing:
tensor([[0.0000, 0.0000, 0.0000, 0.9000, 0.9000, 0.0000, 0.0000, 0.9000],
        [0.0000, 0.0000, 0.0000, 0.9000, 0.9000, 0.0000, 0.0000, 0.9000],
        [0.0000, 0.0000, 0.0000, 0.9000, 0.9000, 0.0000, 0.0000, 0.9000],
        [0.0000, 0.0000, 0.0000, 0.9000, 0.9000, 0.0000, 0.0000, 0.9000],
        [0.9000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9000, 0.0000],
        [0.9000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9000, 0.0000],
        [0.9000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9000, 0.0000],
        [0.0917, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],
       requires_grad=True)

In [29]:
import torch.optim as optim
device = torch.device("cpu")
model = DFALC(params,dataset.conceptSize, dataset.roleSize, cEmb_init=cEmb_init, rEmb_init=rEmb_init, device=device)
dataset.mode = 5
optimizer = optim.Adam(model.parameters(),lr=0.05)
print('TV', model.cEmb[0])
print('screen',model.cEmb[4])
print(model.rEmb[0])
for e in range(30):
    for l,r,n in dataset:
        loss = model([l.unsqueeze(0),r.unsqueeze(0),n.unsqueeze(0)],dataset.mode,device)
        print("loss: ", loss)
        # if loss.item() == 0: break
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # if loss.item() == 0: break
print('TV', model.cEmb[0])
print('screen',model.cEmb[4])
print(model.rEmb[0])
    
    

TV tensor([0.0000, 0.0000, 0.0000, 0.9000, 0.9000, 0.0000, 0.0000, 0.9000],
       grad_fn=<SelectBackward>)
screen tensor([0.9000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9000, 0.0000],
       grad_fn=<SelectBackward>)
tensor([[0.0000, 0.9000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.9000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<SelectBackward>)
loss:  tensor(0.9000, grad_fn=<MeanBackward0>)
loss:  tensor(0.8000, grad_fn=<MeanBackward0>)
loss:  tensor(0.7000, grad_fn=<MeanBackward0>)
loss:  tens

c:\Users\92803\.conda\envs\rl_hw\lib\site-packages\ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':
c:\Users\92803\.conda\envs\rl_hw\lib\site-packages\ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


In [30]:
import torch.optim as optim
device = torch.device("cpu")
model = DFALC(params,dataset.conceptSize, dataset.roleSize, cEmb_init=cEmb_init, rEmb_init=rEmb_init, device=device)
dataset.mode = 6
optimizer = optim.Adam(model.parameters(),lr=0.05)
print('parent', model.cEmb[6])
print('child',model.cEmb[1])
print(model.rEmb[1])
for e in range(30):
    for l,r,n in dataset:
        loss = model([l.unsqueeze(0),r.unsqueeze(0),n.unsqueeze(0)],dataset.mode,device)
        # print("loss: ", loss)
        # if loss.item() == 0: break
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # if loss.item() == 0: break
print('parent', model.cEmb[6])
print('child',model.cEmb[1])
print(model.rEmb[1])
    
    

parent tensor([0.9000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9000, 0.0000],
       grad_fn=<SelectBackward>)
child tensor([0.0000, 0.0000, 0.0000, 0.9000, 0.9000, 0.0000, 0.0000, 0.9000],
       grad_fn=<SelectBackward>)
tensor([[0.0000, 0.9000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.9000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<SelectBackward>)
parent tensor([0.2396, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9000, 0.0000],
       grad_fn=<SelectBackward>)
child tensor([0.4030, 1.0024, 0.4030

c:\Users\92803\.conda\envs\rl_hw\lib\site-packages\ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':
c:\Users\92803\.conda\envs\rl_hw\lib\site-packages\ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
